# Лабараторная работа 3
## Классификация в sklearn. Метрики классификации.
Выполнил: 6131-010402D, **Музыка Артем Алексеевич**

### Выбор и описание набора данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/spotify.csv')

C:\Users\Stormrage\AppData\Local\Temp\ipykernel_6748\810925950.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/spotify.csv')


In [3]:
df.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN


10. Колонки с 0 по 10 являются характеристиками песен, представленнные в `float64` или `int64` типе. 
11. **type** - тип файла в экосистеме spotify. Тип: `object` 
12.  **id** - является буквенно-численным идентификатором песни. Тип: `object`
13.  **uri** - своего рода ссылка на файл. Тип: `object` 
14.  **track_href** своего рода https-ссылка на файл. Тип: `object` 
15.  **analysis_url** своего рода https-ссылка на анализ. Тип: `object` 
16.  **duration_ms** - продолжительность трека в милисекундах. Тип: `int64`
17.  **time_signature** - размер песни. Тип:  `int64`  
18.  **genre** - жанр песни. Будет выступать в роли классов для классификации. Тип: `object`
19.  **song_name** - название песни. Тип: `object`
20.  **Unnamed: 0** - безыменные треки. Тип: `int64`
21.  **title** - название плейлиста в котором хранится трек. Тип: `object` 

### Преобработка наборов данных
Сначала удалим колонки, которые не имеют для нас значения. 

In [4]:
for_delete = ['type','uri','track_href','analysis_url','song_name','Unnamed: 0', 'title', 'id']
for f in for_delete:
    df.pop(f)

Теперь создадим отдельно список жанров и преобразуем его в целочисленный вид.

In [5]:
genres = list(df["genre"])

In [6]:
def repitless(lst):
    new_list = []
    for g in lst:
        if g not in new_list:
            new_list.append(g)
    return new_list

def str_to_int(lst):
    less = repitless(lst)
    for g in range(len(less)):
        for genre in range(len(lst)):
            if lst[genre] == less[g]:
                lst[genre] = g
    return lst

In [7]:
Y = str_to_int(genres)

In [8]:
df.pop('genre')

0        Dark Trap
1        Dark Trap
2        Dark Trap
3        Dark Trap
4        Dark Trap
           ...    
42300    hardstyle
42301    hardstyle
42302    hardstyle
42303    hardstyle
42304    hardstyle
Name: genre, Length: 42305, dtype: object

## 1. Многоклассовая классификация
### 1.1 Простой пример с оценкой accuracy.
Задача состоит в следующем. Делим датасет на тестовую и тренирочную выборки. Обучаем классификатор распозновать жанр на основе характеристик песен. Проверяем результативность. Всего в датасете представлено 15 жанров (классов). В качестве модели используем простой линейный классификатор.

In [10]:
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
X = []
for i in range(len(df)):
    X.append([])
for column in list(df):
    new_list = list(df[column])
    for i in range(len(new_list)):
        X[i].append(new_list[i])


In [12]:
data, labels = shuffle(X, Y)

In [13]:
test_size = int(len(data) * 0.9)
Xtrain = data[:test_size]
Ytrain = labels[:test_size]
Xtest = data[test_size:]
Ytest = labels[test_size:]

In [15]:
clf = LogisticRegression()
clf.fit(Xtrain, Ytrain)
predictions = clf.predict(Xtest)
accuracy_score(Ytest, predictions)

0.21980619238950602

### 1.2 С применением кросс-валидации

### 1.3 Работа с гиперпараметрами

### 1.4 Создание расширенной матрицы ошибок

## 2. Бинарная классификация

## 3. С использованием нормализации признаков для двух датасетов